In [1]:
import snappy
from os.path import join
from glob import glob
import pandas as pd
import numpy as np
import os
import glob
import jpy
System = jpy.get_type('java.lang.System')
System.gc()
import gc

import re
from geomet import wkt
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
from snappy import jpy
HashMap = snappy.jpy.get_type('java.util.HashMap')
import time
import evros
import configparser

<h1> Read the Location Where the Sentinels are Stored </h1>

In [5]:
# Parse the ini file
config = configparser.ConfigParser()
config.read('sentinels.ini')
path = config['SENTINELS']['path']

<h1> Sentinel 2 </h1>

# Read Products

In [6]:
# Set target folder and extract metadata - Change accordingly 
# product_path = str('D://EVROS//Satellite_Images//Copernicus_2016_2020//CODE//Sample_Data//raw/Sentinel2/')

product_path = str(path)+str('raw/Sentinel2/')

input_S2_files = sorted(os.listdir(product_path))

name, process_level, sensing_date, relevant_orbit, height, width, band_names = ([] for i in range(7))

for i in input_S2_files:
    process_level.append(i.split("_")[1])
    sensing_date.append(i.split("_")[2])
    relevant_orbit.append(i.split("_")[4])
    # Read with snappy
    s2_read = snappy.ProductIO.readProduct(product_path + i)
    name.append(s2_read.getName())
    height.append(s2_read.getSceneRasterHeight())
    width.append(s2_read.getSceneRasterWidth())
    band_names.append(s2_read.getBandNames())
    
df_s2_read = pd.DataFrame({'Name': name, 'Process_Level': process_level, 'Sensing_Date': sensing_date, 'Relevant_Orbit': relevant_orbit, 'Height': height, 'Width': width, 'Band_Names': band_names})
display(df_s2_read)

,Name,Process_Level,Sensing_Date,Relevant_Orbit,Height,Width,Band_Names
0,S2A_MSIL2A_20200111T091341_N9999_R050_T35TMF_2...,MSIL2A,20200111T091341,R050,10980,10980,"[B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11,..."
1,S2A_MSIL2A_20200417T085601_N9999_R007_T35TMF_2...,MSIL2A,20200417T085601,R007,10980,10980,"[B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11,..."
2,S2A_MSIL2A_20200716T085601_N0214_R007_T35TMF_2...,MSIL2A,20200716T085601,R007,10980,10980,"[B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11,..."
3,S2A_MSIL2A_20201024T090041_N0214_R007_T35TMF_2...,MSIL2A,20201024T090041,R007,10980,10980,"[B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11,..."


<h1> Produce Classification Maps from Sentinel 2 </h1>

In [7]:
# outpath_name = 'D://EVROS//Satellite_Images//Copernicus_2016_2020//CODE//Sample_Data/classification_maps/'

outpath_name = str(path)+str('classification_maps/')

if not(os.path.exists(outpath_name) and os.path.isdir(outpath_name)):
    os.makedirs(outpath_name)

for i in input_S2_files:
    
    s2_read = snappy.ProductIO.readProduct(product_path + i)                   
    name = s2_read.getName()
    print(f'Reading {name}\n')
    output = str(outpath_name) + str(name)
    
    # subset the product
    subset_prod = evros.subset_s2(s2_read)                                           

    # resample the product to 10m spatial resolution
    resampled = evros.resample_s2(subset_prod)                                       
     
    # calculate the first principal component
    component_1 = evros.pca_s2(resampled)
    
    # calculate the NDVI and mNDVI
    ndvi, ndwi = evros.ndvi_ndwi(resampled)
    
    # stack the pca , ndvi and mndvi into a single product
    stacked = evros.stack(component_1, ndvi, ndwi)
    
    # implement Kmeans unsupervised classification
    classification_map = evros.kmeans_s2(stacked)
    
    
    # Wite the classification map on the local disk
    snappy.ProductIO.writeProduct(classification_map , output+"_cluster", 'GeoTIFF')  # save the new product on disk
    print(f'Saving the new product on disk...')
    
    del s2_read, subset_prod, resampled, component_1, ndvi, ndwi, stacked
    gc.collect()
    print('\n')
    
print('Calculating Classification Maps from Sentinel 2 Images has Finished')

Reading S2A_MSIL2A_20200111T091341_N9999_R050_T35TMF_20210628T075932

Subseting the product....
Resampling the product....
Applying PCA.....
Calculating NDVI and mNDWI....
Stacking the images....
Implementing Kmeans....
Saving the new product on disk...


Reading S2A_MSIL2A_20200417T085601_N9999_R007_T35TMF_20210628T081551

Subseting the product....
Resampling the product....
Applying PCA.....
Calculating NDVI and mNDWI....
Stacking the images....
Implementing Kmeans....
Saving the new product on disk...


Reading S2A_MSIL2A_20200716T085601_N0214_R007_T35TMF_20200716T120405

Subseting the product....
Resampling the product....
Applying PCA.....
Calculating NDVI and mNDWI....
Stacking the images....
Implementing Kmeans....
Saving the new product on disk...


Reading S2A_MSIL2A_20201024T090041_N0214_R007_T35TMF_20201024T114948

Subseting the product....
Resampling the product....
Applying PCA.....
Calculating NDVI and mNDWI....
Stacking the images....
Implementing Kmeans....
Saving the n

<h1> Sentinel 1 </h1>

<h1> Read Products </h1>

In [8]:
# Set target folder and extract metadata - Change accordingly
# product_path = str('D://EVROS//Satellite_Images//Copernicus_2016_2020//CODE//Sample_Data//raw/Sentinel1/')
product_path = str(path)+str('raw/Sentinel1/')

input_S1_files = sorted(os.listdir(product_path))

name, process_level, sensing_date, relevant_orbit, height, width, band_names = ([] for i in range(7))


for i in input_S1_files:
    process_level.append(i.split("_")[1])
    sensing_date.append(i.split("_")[2])
    relevant_orbit.append(i.split("_")[4])
    # Read with snappy
    s1_read = snappy.ProductIO.readProduct(product_path + i)
    name.append(s1_read.getName())
    height.append(s1_read.getSceneRasterHeight())
    width.append(s1_read.getSceneRasterWidth())
    band_names.append(s1_read.getBandNames())
    
df_s1_read = pd.DataFrame({'Name': name, 'Process_Level': process_level, 'Sensing_Date': sensing_date, 'Relevant_Orbit': relevant_orbit, 'Height': height, 'Width': width, 'Band_Names': band_names})
display(df_s1_read)



,Name,Process_Level,Sensing_Date,Relevant_Orbit,Height,Width,Band_Names
0,S1A_IW_GRDH_1SDV_20200115T042244_20200115T0423...,IW,GRDH,20200115T042244,16691,25972,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."
1,S1A_IW_GRDH_1SDV_20200414T161600_20200414T1616...,IW,GRDH,20200414T161600,16671,26450,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."
2,S1A_IW_GRDH_1SDV_20200713T042249_20200713T0423...,IW,GRDH,20200713T042249,16691,25967,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."
3,S1A_IW_GRDH_1SDV_20201017T042253_20201017T0423...,IW,GRDH,20201017T042253,16691,25972,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."


<h1> Produce Classification Maps using Sentinel 1 </h1>

In [9]:
outpath_name = str(path)+str('classification_maps/')

if not(os.path.exists(outpath_name) and os.path.isdir(outpath_name)):
    os.makedirs(outpath_name)

for i in input_S1_files:
    
    s1_read = snappy.ProductIO.readProduct(product_path + i)                   # read the product
    name = s1_read.getName()
    output = str(outpath_name) + str(name)
    print(f'Reading product {name}\n')
    
    # Apply orbit file
    appply_orbit = evros.applyorbitfile(s1_read)
    
    # Callibration
    calibration = evros.s1_calibration(appply_orbit)
    
    # Speckle Filtering
    speckle = evros.specklefilter(calibration)
    
    # Linear to dB
    db = evros.lineartodB(speckle)
    
    # Terrain Correction
    terrain = evros.terrain_correction(db)
    
    # Apply resampling
#     resampled = evros.resample_s1(terrain)
    
    # Subset
    subset = evros.subset_s1(terrain)
    
    # Applying Kmeans
    cluster = evros.kmeans_s1(subset)
    
    
    # Saving the product on local disk
    snappy.ProductIO.writeProduct(cluster , output+"_cluster", 'GeoTIFF')  
    print(f'Saving the new product on disk...')
    
    del s1_read, appply_orbit, calibration, speckle, terrain, subset
    gc.collect()
    print('\n')
    
print('Calculating Classification Maps from Sentinel 1 products has finished!')

Reading product S1A_IW_GRDH_1SDV_20200115T042244_20200115T042309_030806_0388A1_15AA
Orbit updated succesfully
Calibration implemented succesfully
Speckle Filtering applied succesfully
Applying Linear to dB...
Terrain Correction and Reprojection applied successfully
Subset implemented succesfully
Sigma0_VH_db,Sigma0_VV_db
Applying K-means has finished
Saving the new product on disk...


Reading product S1A_IW_GRDH_1SDV_20200414T161600_20200414T161625_032126_03B6B8_3C7F
Orbit updated succesfully
Calibration implemented succesfully
Speckle Filtering applied succesfully
Applying Linear to dB...
Terrain Correction and Reprojection applied successfully
Subset implemented succesfully
Sigma0_VH_db,Sigma0_VV_db
Applying K-means has finished
Saving the new product on disk...


Reading product S1A_IW_GRDH_1SDV_20200713T042249_20200713T042314_033431_03DFB3_BC6C
Orbit updated succesfully
Calibration implemented succesfully
Speckle Filtering applied succesfully
Applying Linear to dB...
Terrain Corre

<h1> Align Classification Maps </h1>

In [10]:
from osgeo import gdal
import numpy as np
import os
from glob import glob
from gdal import gdalconst
os.chdir(str(path)+str('classification_maps/')) # Change

In [12]:
# Create a list of all available sentinel 1 images
listofimages = []
listofimages = glob('*.tif')

print(f'The master image is {listofimages[0]}\n')

for image in listofimages[1:]:
    print(image) # print the name of the product to be aligned 
    
    # Slave Product
    inputimage = gdal.Open(image, gdalconst.GA_ReadOnly)
    inputProj = inputimage.GetProjection()

    #Master product - All images will be aligned with this one
    reference = gdal.Open(listofimages[0], gdalconst.GA_ReadOnly)
    referenceProj = reference.GetProjection()
    referenceTrans = reference.GetGeoTransform()
    x = reference.RasterXSize
    y = reference.RasterYSize

    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(str(image[:-4])+str('_aligned.tif'), x, y, 2, gdal.GDT_Float32)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    dstband= output.GetRasterBand(1)
    dstband.SetNoDataValue(np.nan)
    gdal.ReprojectImage(inputimage, output, inputProj, referenceProj, gdalconst.GRA_Med)
    dstband.FlushCache()
    output.FlushCache()

    inputimage = None
    reference = None
    dstband = None
    output = None
    
print('Aligning Images has Finished!')

The master image is S1A_IW_GRDH_1SDV_20200115T042244_20200115T042309_030806_0388A1_15AA_cluster.tif

S1A_IW_GRDH_1SDV_20200414T161600_20200414T161625_032126_03B6B8_3C7F_cluster.tif
S1A_IW_GRDH_1SDV_20200713T042249_20200713T042314_033431_03DFB3_BC6C_cluster.tif
S1A_IW_GRDH_1SDV_20201017T042253_20201017T042318_034831_040F61_EB8E_cluster.tif
S2A_MSIL2A_20200111T091341_N9999_R050_T35TMF_20210628T075932_cluster.tif
S2A_MSIL2A_20200417T085601_N9999_R007_T35TMF_20210628T081551_cluster.tif
S2A_MSIL2A_20200716T085601_N0214_R007_T35TMF_20200716T120405_cluster.tif
S2A_MSIL2A_20201024T090041_N0214_R007_T35TMF_20201024T114948_cluster.tif
Aligning Images has Finished!


<h1> Create the Final Probability Map </h1>

In [13]:
# The name of the folder where the probability map will be stored.
# outputPath = 'D://EVROS//Satellite_Images//Copernicus_2016_2020//CODE//Sample_Data/probability_maps/'

outputPath = str(path)+str('probability_maps/')
if not(os.path.exists(outputPath) and os.path.isdir(outputPath)):
    os.makedirs(outputPath)
    
    
listofclusters = []
listofclusters = glob('S**_aligned.tif')


##########################  ############################
# Get array image dimensions, projection, transform
im = gdal.Open(listofclusters[0])
Y = im.RasterYSize
X = im.RasterXSize
proj = im.GetProjection()
transf = im.GetGeoTransform()
im = None

##############  stack classification maps from previous step
print('Costructing Classification map....\n')
stack = np.empty((X*Y, len(listofclusters)))
c = 0
for i in listofclusters:
    im = gdal.Open(i)
    img = im.GetRasterBand(1).ReadAsArray()
    stack[:, c] = img.flatten()
    c += 1
# # Replace value 0 (clouds) with NaN    
stack[stack >1 ] = np.nan

# mean
meanm = np.nanmean(stack,1)
stack = None
# Export Map
out_dat = meanm.reshape((Y, X))
driverTiff = gdal.GetDriverByName('GTiff')
out = driverTiff.Create(outputPath+'Mean_Map_Final.tif', X, Y, 1, gdal.GDT_Float32)
out.SetGeoTransform(transf)
out.SetProjection(proj)
out.GetRasterBand(1).WriteArray(out_dat)
out = None

print('Calculating the Final Probability Map has Finished!!')

Costructing Classification map....

Calculating the Final Probability Map has Finished!!
